In [1]:
from exif_tagger.database import FaceDatabase,UNKNOWN_PERSON,Face,Person,Picture
from exif_tagger.face_recognition import FaceRecognitionModel
from pathlib import Path
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [2]:
DEFAULT_DB_PATH = Path("/home/pierre/repositories/exif-tagger/db/face-database.sqlite")
DEFAULT_UNKNOWN_EMBEDDING_PATH = Path(
    "/home/pierre/repositories/exif-tagger/res/embeddings_vg2.npy"
)
DEFAULT_FACE_RECOGNITION_PATH = Path(
    "/home/pierre/repositories/exif-tagger/models/face-recognition.joblib"
)

db = FaceDatabase(DEFAULT_DB_PATH)
fc = FaceRecognitionModel.from_file(model_filename=DEFAULT_FACE_RECOGNITION_PATH)

Running on device: cpu


In [ ]:

fc.detect_faces("/home/pierre/repositories/exif-tagger/test/test_big/PXL_20231109_034149044.jpg")

In [2]:
from exif_tagger.embeddings import generate_embeddings
embs,names = generate_embeddings("/home/pierre/Downloads/VGG-Face2/data/vggface2_test/small_test")

Running on device: cpu


  0%|          | 0/11 [00:00<?, ?it/s]

ALIGNED 100 torch.Size([3, 160, 160])
STACKED torch.Size([100, 3, 160, 160])
tensor([[[-0.3320, -0.2852, -0.1836,  ...,  0.2695,  0.3008,  0.3164],
         [-0.2617, -0.1992, -0.0664,  ...,  0.2852,  0.3164,  0.3320],
         [-0.1914, -0.1133,  0.0508,  ...,  0.3008,  0.3398,  0.3555],
         ...,
         [-0.1680, -0.1445, -0.0977,  ...,  0.3477,  0.2773,  0.2383],
         [-0.0273, -0.0039,  0.0273,  ...,  0.3711,  0.3320,  0.3164],
         [ 0.1133,  0.1211,  0.1367,  ...,  0.3945,  0.3867,  0.3867]],

        [[-0.7070, -0.6445, -0.5273,  ...,  0.1055,  0.1211,  0.1289],
         [-0.6367, -0.5586, -0.4023,  ...,  0.1133,  0.1289,  0.1367],
         [-0.5664, -0.4648, -0.2695,  ...,  0.1133,  0.1289,  0.1367],
         ...,
         [-0.7930, -0.7695, -0.7227,  ..., -0.7773, -0.8320, -0.8555],
         [-0.6836, -0.6680, -0.6211,  ..., -0.7695, -0.7852, -0.7930],
         [-0.5664, -0.5586, -0.5352,  ..., -0.7539, -0.7383, -0.7305]],

        [[-0.7227, -0.6523, -0.5195,  .

In [3]:
unknown_embeddings = np.load(DEFAULT_UNKNOWN_EMBEDDING_PATH)
unknown_embeddings_train = unknown_embeddings[:len(unknown_embeddings)//2]
unknown_embeddings_test = unknown_embeddings[len(unknown_embeddings)//2:]
unknown_embeddings.shape

(169218, 512)

In [4]:
db.getPersons()

[Person(name='Unknown', surname='Unknown', id=1),
 Person(name='Pierre', surname='Louvart', id=2),
 Person(name='Kerian', surname='Tiennot', id=3)]

In [5]:
pierre = db.getPersonsByNameSurname("Pierre", "Louvart")[0]
pierre_embeddings = np.array(db.getEmbeddingsForPerson(pierre.id))
pierre_embeddings.shape
pierre_embeddings_train = pierre_embeddings[::2]
pierre_embeddings_test = pierre_embeddings[1::2]
pierre_embeddings.shape

(79, 512)

In [6]:
kerian = db.getPersonsByNameSurname("Kerian", "Tiennot")[0]
kerian_embeddings = np.array(db.getEmbeddingsForPerson(kerian.id))
kerian_embeddings.shape
kerian_embeddings_train = kerian_embeddings[::2]
kerian_embeddings_test = kerian_embeddings[1::2]
kerian_embeddings.shape

(11, 512)

In [ ]:
people_embeddings = np.array(db.getEmbeddingsForPerson(UNKNOWN_PERSON.id))
people_embeddings.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

X = np.concatenate([unknown_embeddings_train, kerian_embeddings_train])
Y = np.concatenate([np.zeros(len(unknown_embeddings_train)), 1*np.ones(len(kerian_embeddings_train))])
# X = np.concatenate([unknown_embeddings_train, pierre_embeddings_train, kerian_embeddings_train])
# Y = np.concatenate([np.zeros(len(unknown_embeddings_train)), np.ones(len(pierre_embeddings_train)), 2*np.ones(len(kerian_embeddings_train))])


model.fit(X, Y)
model

In [ ]:
np.mean(
    model.predict(X[::50]) == Y[::50]
)

In [ ]:
model.predict(pierre_embeddings_test)

In [ ]:
model.predict(pierre_embeddings_train)

In [ ]:
model.predict(kerian_embeddings_test)

In [ ]:
model.predict(unknown_embeddings_train[::100])

In [ ]:
X = np.concatenate([kerian_embeddings, embs, pierre_embeddings[:20]])
print(pierre_embeddings.shape)
print(embs.shape)
print(kerian_embeddings.shape)
plt.figure(figsize=(16, 8))
plt.imshow(X)

In [ ]:
X = np.concatenate([embs, unknown_embeddings[::100]])
print(pierre_embeddings.shape)
print(embs.shape)
print(kerian_embeddings.shape)
plt.figure(figsize=(16, 8))
plt.imshow(X)

In [ ]:
unknown_embeddings.shape,  embs.shape

In [ ]:
np.cov(X)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from exif_tagger.database import FaceDatabase,UNKNOWN_PERSON,Face,Person,Picture
import PIL.Image
import io


imgs = db.getImagesForPerson(pierre.id)
widget_imgs = []
for img in imgs:
    bin = io.BytesIO()
    img.save(bin, format='JPEG')
    widget_imgs.append(widgets.Image(value=bin.getvalue()))



widgets.GridBox(
    children = widget_imgs,
    layout=widgets.Layout(
        grid_template_columns='256px ' * 5,
    )
)

In [ ]:
imgs = db.getImagesForPerson(UNKNOWN_PERSON.id)
imgs

In [ ]:
t = PIL.Image.fromarray(np.array(imgs[0]).astype(np.uint8).transpose(1,2,0))
t

In [ ]:


model = PCA(n_components=2)
model = model.fit(X)

plt.scatter(*model.transform(unknown_embeddings).T, color="b")
plt.scatter(*model.transform(pierre_embeddings_test).T, color="r")
plt.scatter(*model.transform(pierre_embeddings_train).T, color="g")
plt.scatter(*model.transform(people_embeddings).T, color="orange")